In [ ]:
!pip3 install skfeature-chappers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.utils import resample
from skfeature.function.similarity_based import fisher_score
import numpy as np
import pandas as pd

In [ ]:
!pip3 install matplotlib
import random
import math,time,sys, os
from matplotlib import pyplot
from datetime import datetime

In [ ]:
dataframe_train=pd.read_csv('PAMPA_Train_Feature.csv')
#dataframe_train.drop(dataframe_train.tail(1).index,inplace=True)
dataframe_train = dataframe_train.iloc[: , 1:]
print(dataframe_train)
y_train=pd.read_csv('PAMPA2_Train_Label.csv')
y_train

In [ ]:
coln=dataframe_train.columns
coln1=y_train.columns

In [ ]:
dataframe_test=pd.read_csv('PAMPA2_Test_Feature.csv')
#dataframe_test.drop(dataframe_test.tail(1).index,inplace=True)
dataframe_test = dataframe_test.iloc[: , 1:]
print(dataframe_test)
y_test=pd.read_csv('PAMPA2_Test_Label.csv')
y_test

In [ ]:
dataframe_test.columns=coln
y_test.columns=coln1

In [ ]:
dataframe=dataframe_train.append(dataframe_test,ignore_index=True)
print(dataframe)
y=y_train.append(y_test,ignore_index=True)
y


In [ ]:
cols=[]
for i in range(len(dataframe.columns)):
  col1=dataframe[dataframe.columns[i]].to_numpy()
  if(np.sum(col1)==0):
    cols.append(dataframe.columns[i])

dataframe = dataframe.drop(columns=cols)
print(dataframe.shape[1])

In [ ]:
df=dataframe.copy()
df['labels']=y
df

In [ ]:
x=df.copy()

In [ ]:
ln=df.shape[1]
str=df.columns[ln-1]
print(df)

In [ ]:
x=x.drop([str],axis=1)
dataset=x.to_numpy()
dataset

In [ ]:
def sort_agents(agents, obj,train_X,val_X,train_Y,val_Y):
    # sort the agents according to fitness
    #train_X, val_X, train_Y, val_Y = data.train_X, data.val_X, data.train_Y, data.val_Y
    (obj_function, weight_acc) = obj

    # if there is only one agent
    if len(agents.shape) == 1:
        num_agents = 1
        fitness = obj_function(agents, train_X, val_X, train_Y, val_Y, weight_acc)
        return agents, fitness

    # for multiple agents
    else:
        num_agents = agents.shape[0]
        fitness = np.zeros(num_agents)
        for id, agent in enumerate(agents):
            fitness[id] = obj_function(agent, train_X, val_X, train_Y, val_Y, weight_acc)
            
        idx = np.argsort(-fitness)
        sorted_agents = agents[idx].copy()
        sorted_fitness = fitness[idx].copy()
        
    return sorted_agents, sorted_fitness


In [ ]:
def compute_fitness(agent, train_X, test_X, train_Y, test_Y, weight_acc=0.99):
    # compute a basic fitness measure
    if(weight_acc == None):
        weight_acc = 0.99
    
    weight_feat = 1 - weight_acc
    num_features = len(agent)
   
    acc = compute_accuracy(agent, train_X, test_X, train_Y, test_Y)
    feat = (num_features - np.sum(agent))/num_features

    fitness = weight_acc * acc + weight_feat * feat
    
    return fitness


In [ ]:
from sklearn.feature_selection import f_regression

# inputs:
#    X: pandas.DataFrame, features
#    yy: pandas.Series, target variable
#    K: number of features to select


X=x
yy=df[str].to_numpy()
K=x.shape[1]

# compute F-statistics and initialize correlation matrix
F = pd.Series(f_regression(X, yy)[0], index = X.columns)
corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

# initialize list of selected features and list of excluded features
selected = []
not_selected = X.columns.to_list()
Dict={}
# repeat K times
for i in range(K):
  
    # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
    if i > 0:
        last_selected = selected[-1]
        corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
        
    # compute FCQ score for all the (currently) excluded features (this is Formula 2)
    score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
    
    # find best feature, add it to selected and remove it from not_selected
    # print(score[score.argmax()])
    # print(score.max())
    # print(score.index[score.argmax()])
    best = score.index[score.argmax()]
    selected.append(best)
    not_selected.remove(best)
    Dict[best]=score.max()

mrMr=[]
for i in range(x.shape[1]):
  mrMr.append(Dict[x.columns[i]])
#print(mrMr)  

In [ ]:
for i in range(len(mrMr)):
  mrMr[i]=mrMr[i]/(max(mrMr))

In [ ]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [ ]:
!pip3 install Py-FS
def fitness_score_values(data,target):
    from Py_FS.filter import PCC as PCC
    from Py_FS.filter import MI as MI
    from sklearn.feature_selection import mutual_info_classif
    from Py_FS.filter import Relief as Relief
    from sklearn.feature_selection import chi2
    
    print(data)
    print(target)
    #print('Pcc scores')
    #PCC_solution = PCC(data, target)
    #print(PCC_solution.scores)
    print('MI scores')
    mi_scores=mutual_info_classif(data, target, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
    print((mi_scores))
    
    print('Relief scores')
    Relief_solution = Relief(data, target)
    print((Relief_solution.scores))
    
    
        
   
    shapley=[]
    b=x.shape[1]
    print(b)
    for i in range(b):
        shapley.append((mi_scores[i]+mrMr[i]+Relief_solution.scores[i])/3)
    return shapley

In [ ]:
target=df[str].to_numpy()
print(target)

In [ ]:
!pip3 install tabulate
!pip3 install ReliefF
score=fitness_score_values(x.values,target)

In [ ]:
def compute_accuracy(agent, train_X, test_X, train_Y, test_Y): 
    # compute classification accuracy of the given agents
    fitness=0
    number=0
    # print(len(agent))
    # print(len(score))
    for i in range(len(agent)):
        if agent[i]==1:
            number=number+1
            fitness=fitness+score[i]
           
    if number==0:
      return 0
    return fitness/number        

In [ ]:
def cal(agent): 
    # compute classification accuracy of the given agents
    fitness=0
    number=0
    for i in range(len(agent)):
        if agent[i]==1:
            number=number+1
            fitness=fitness+score[i]
           
    if number==0:
     return 0,0
    return fitness,number  

In [ ]:
def initialize(popSize,dim):
	population=np.zeros((popSize,dim))
	minn = 1
	maxx = math.floor(0.8*dim)
	if maxx<minn:
		minn = maxx

	for i in range(popSize):
		random.seed(i**3 + 10 + time.time() ) 
		no = random.randint(minn,maxx)
		if no == 0:
			no = 1
		random.seed(time.time()+ 100)
		pos = random.sample(range(0,dim-1),no)
		for j in pos:
			population[i][j]=1

		# print(population[i])  
	return population

 
def fitness(solution, trainX, trainy, testX,testy):
	cols=np.flatnonzero(solution)
	val=1
	if np.shape(cols)[0]==0:
		return val	
	clf=KNeighborsClassifier(n_neighbors=5)
	train_data=trainX[:,cols]
	test_data=testX[:,cols]
	clf.fit(train_data,trainy)
	error=1-clf.score(test_data,testy)

	#in case of multi objective  []
	featureRatio = (solution.sum()/np.shape(solution)[0])
	val=omega*error+(1-omega)*featureRatio
	# print(error,featureRatio,val)
	return val

def allfit(population, trainX,  trainy,testX, testy):
	x=np.shape(population)[0]
	acc=np.zeros(x)
	for i in range(x):
		acc[i]=fitness(population[i],trainX,trainy,testX,testy)     
		#print(acc[i])
	return acc


def selectParentRoulette(popSize,fitness):
    # Perform roulette wheel selection
    maximum = sum([f for f in fitness])
    selection_probs = [f/maximum for f in fitness]
    return np.random.choice(len(fitness), p=selection_probs)


def randomwalk(agent,agentFit):
	percent = 30
	percent /= 100
	neighbor = agent.copy()
	size = np.shape(agent)[0]
	upper = int(percent*size)
	if upper <= 1 or upper>size:
		upper = size
	x = random.randint(1,upper)
	pos = random.sample(range(0,size - 1),x)
	for i in pos:
		neighbor[i] = 1 - neighbor[i]
	return neighbor

def adaptiveBeta(agent,agentFit, trainX, trainy,testX,testy):
	bmin = 0.1 #parameter: (can be made 0.01)
	bmax = 1
	maxIter = 10 # parameter: (can be increased )
	maxIter = int(max(10,10*agentFit))


	for curr in range(maxIter):
		neighbor = agent.copy()
		size = np.shape(agent)[0]
		neighbor = randomwalk(neighbor,agentFit)

		beta = bmin + (curr / maxIter)*(bmax - bmin)
		for i in range(size):
			random.seed( time.time() + i )
			if random.random() <= beta:
				neighbor[i] = agent[i]
		neighFit = fitness(neighbor,trainX,trainy,testX,testy)
		if neighFit <= agentFit:
			agent = neighbor.copy()
			agentFit = neighFit
	return (agent,agentFit)

#============================================================================
def geneticAlgo(dataset,popSize,maxIter,randomstate):

	#--------------------------------------------------------------------
	(a,b)=np.shape(df)
	print(a,b)
	data = df.values[:,0:b-1]
	#print(data)
	label=target

  

	print(label)
	dimension = np.shape(data)[1] #solution dimension
	#---------------------------------------------------------------------

	# cross = 5
	# test_size = (1/cross)
	per=0.3
	trainX, testX, trainy, testy = train_test_split(data, label ,stratify=label ,test_size=per,random_state=42) 

  
	x_axis = []
	y_axis = []
	population = initialize(popSize,dimension)
  
	print(population)
	GBESTSOL = np.zeros(np.shape(population[0]))
	GBESTFIT = -1000



	

	start_time = datetime.now()
  
	for currIter in range(1,maxIter):
		newpop = population
		#newpop = np.zeros((popSize,dimension))
	

		obj_function=compute_fitness
		weight_acc = None
		obj=(obj_function, weight_acc)
	  
		population,fitList=sort_agents(population,obj,trainX,trainy,testX,testy)
	
	  

		bestInx = np.argmax(fitList)
		fitBest = max(fitList)
		print(currIter,'best:',fitBest,population[bestInx].sum())
	  
		# print(population[bestInx])
		if fitBest > GBESTFIT:
			GBESTSOL = population[bestInx].copy()
			GBESTFIT = fitBest
		#print(fitList)
		for selectioncount in range(int(popSize/2)):
			parent1 = selectParentRoulette(popSize,fitList)
			parent2 = parent1
			while parent2 == parent1:
				random.seed(time.time())
				# parent2 = random.randint(0,popSize-1)
				parent2 = selectParentRoulette(popSize,fitList)

				# print(parent2)
			# print('parents:',parent1,parent2)
			parent1 = population[parent1].copy()
			parent2 = population[parent2].copy()
			#cross over between parent1 and parent2
			child1 = parent1.copy()
			child2 = parent2.copy()
			for i in range(dimension):
				random.seed(time.time())
				if random.uniform(0,1)<crossoverprob:
					child1[i]=parent2[i]
					child2[i]=parent1[i]
			i = selectioncount
			j = int(i+(popSize/2))
			# print(i,j)
			newpop[i]=child1.copy()
			newpop[j]=child2.copy()
	 

    
		mutationprob = muprobmin + (muprobmax - muprobmin)*(currIter/maxIter)
		for i in range(popSize):
			for k in range(1):
				fit,no=cal(newpop[i])
				for j in range(dimension):
					if random.uniform(0,1)>mutationprob:
					 	continue		
					if no==0 or newpop[i][j]==0 and score[j]>(fit/no):
						newpop[i][j]=1
						fit=fit+score[j]
						no=no+1
					if newpop[i][j]==1 and score[j]<(fit/no):
						newpop[i][j]=0
						fit=fit-score[j]
						no=no-1
	  
		

      
    

		population = newpop.copy()

	# pyplot.plot(x_axis,y_axis)
	# pyplot.show()

	#test accuracy
	cols = np.flatnonzero(GBESTSOL)
	val = 1
	if np.shape(cols)[0]==0:
		return GBESTSOL
	clf = KNeighborsClassifier(n_neighbors=5)
	train_data = trainX[:,cols]
	test_data = testX[:,cols]
	clf.fit(train_data,trainy)
	val = clf.score(test_data,testy)
	return GBESTSOL,val,test_data,testy,train_data,trainy

In [ ]:
popSize = 5
maxIter = 5
omega = 0.9
crossoverprob = 0.8
muprobmin = 0.01
muprobmax = 0.05
datasetList = ["Dataset"]
randomstateList=[15,5,15,26,12,7,10,8,37,19,35,2,49,26,1,25,47,12]

In [ ]:
def ga_final_vector():
  for datasetinx in range(1): 
    dataset=datasetList[datasetinx]
    best_accuracy = -100
    best_no_features = 100
    best_answer = []
    accuList = []
    featList = []
    for count in range(15):
      if (dataset == "WaveformEW" or dataset == "KrvskpEW") : #and count>2 :
        break
      print(count)
      answer,testAcc,testX,testy,trainX,trainy = geneticAlgo(dataset+".csv",popSize,maxIter,randomstateList[datasetinx])
      print(testAcc,answer.sum())
      accuList.append(testAcc)
      featList.append(answer.sum())
      if testAcc>=best_accuracy and answer.sum()<best_no_features:
        best_accuracy = testAcc
        best_no_features = answer.sum()
        best_answer = answer.copy()
        testX_final=testX
        testy_final=testy
        trainX_final=trainX
        trainy_final=trainy
      if testAcc>best_accuracy:
        best_accuracy = testAcc
        best_no_features = answer.sum()
        best_answer = answer.copy()
        testX_final=testX
        testy_final=testy
        trainX_final=trainX
        trainy_final=trainy



    print(dataset,"best:",best_accuracy,best_no_features,best_answer)
    return best_answer,trainX_final,trainy_final,testX_final,testy_final

In [ ]:
ga,trainX_final,trainy_final,testX_final,testy_final=ga_final_vector()

In [ ]:
!pip3 install seaborn
import seaborn as sn
import matplotlib.pyplot as plt

classes_= np.array(['Nordic_walking', 'ascending_stairs', 'cycling','descending_stairs', 'ironing','lying', 
                   'rope_jumping', 'running','sitting', 'standing', 'vacuum_cleaning', 'walking'], dtype=object)
!pip3 install mlxtend
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(trainX_final,trainy_final)
y_pred = clf.predict(testX_final)
print(y_pred)
mat = confusion_matrix(testy_final, y_pred)

# pd.DataFrame(testy_final).to_csv("testy.csv")
# pd.DataFrame(y_pred).to_csv("y_pred.csv")
# print(mat)
res=sn.heatmap(mat, cmap='Greens', annot=True, fmt='d')

plt.yticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5], classes_,rotation=0,fontsize=10)
plt.xticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,9.5,10.5,11.5], classes_,rotation=80,fontsize=10)


plt.title('Confusion Matrix',fontsize=20)
plt.xlabel("Predicted Label",fontsize=17)
plt.ylabel("True Label",fontsize=17)
plt.figure(figsize=(120,120))
plt.show()

plt.close()


In [ ]:
from sklearn.metrics import classification_report
classes_= ['Nordic_walking', 'ascending_stairs', 'cycling','descending_stairs', 'ironing','lying', 
                   'rope_jumping', 'running','sitting', 'standing', 'vacuum_cleaning', 'walking']
clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(trainX_final,trainy_final)
y_pred = clf.predict(testX_final)
report=classification_report(testy_final,y_pred,target_names=classes_)
print(report)